In [1]:
import pandas as pd
import json

In [3]:
df = pd.read_csv("data/mentions.csv")
df

,GLOBALEVENTID,EventTimeDate,MentionTimeDate,MentionType,MentionSourceName,MentionIdentifier,SentenceID,Actor1CharOffset,Actor2CharOffset,ActionCharOffset,InRawText,Confidence,MentionDocLen,MentionDocTone,MentionDocTranslationInfo,Extras
0,803003097,20181116020000,20181116124500,1,dw.com,https://www.dw.com/en/opinion-hungarys-viktor-...,5,807,-1,801,1,100,4269,-4.788732,NaN,NaN
1,803111937,20181116124500,20181116124500,1,dw.com,https://www.dw.com/en/opinion-hungarys-viktor-...,19,4383,-1,4411,1,100,4269,-4.788732,NaN,NaN
2,803111940,20181116124500,20181116124500,1,dw.com,https://www.dw.com/en/opinion-hungarys-viktor-...,5,682,807,790,1,100,4269,-4.788732,NaN,NaN
3,803090787,20181116110000,20181116134500,1,dw.com,https://www.dw.com/en/khmer-rouge-leaders-foun...,3,163,172,201,1,100,277,-12.068966,NaN,NaN
4,803030906,20181116051500,20181116134500,1,dw.com,https://www.dw.com/en/khmer-rouge-leaders-foun...,2,101,-1,121,1,100,277,-12.068966,NaN,NaN
5,803117929,20181116131500,20181116131500,1,dw.com,https://www.dw.com/overlay/media/en/cracking-d...,1,245,154,220,1,100,303,-6.521739,NaN,NaN
6,803107988,20181116123000,20181116123000,1,dw.com,https://www.dw.com/en/khmer-rouge-genocide-in-...,10,2522,2503,2491,1,100,12181,-7.120419,NaN,NaN
7,803108279,20181116123000,20181116123000,1,dw.com,https://www.dw.com/en/khmer-rouge-genocide-in-...,10,2503,2522,2491,1,100,12181,-7.120419,NaN,NaN
8,803108376,20181116123000,20181116123000,1,dw.com,https://www.dw.com/en/khmer-rouge-genocide-in-...,3,640,654,690,1,100,12181,-7.120419,NaN,NaN
9,803108556,20181116123000,20181116123000,1,dw.com,https://www.dw.com/en/khmer-rouge-genocide-in-...,29,5524,5624,5586,1,100,12181,-7.120419,NaN,NaN


In [4]:
# sanity check : we only have articles (MentionType 1)
df[df.MentionType!=1]

,GLOBALEVENTID,EventTimeDate,MentionTimeDate,MentionType,MentionSourceName,MentionIdentifier,SentenceID,Actor1CharOffset,Actor2CharOffset,ActionCharOffset,InRawText,Confidence,MentionDocLen,MentionDocTone,MentionDocTranslationInfo,Extras


In [5]:
# distribution of nb of articles by sources
nb_articles = df.groupby("MentionSourceName").size()
nb_articles.describe()

count      50.000000
mean      776.880000
std       540.196927
min       362.000000
25%       428.750000
50%       551.500000
75%       874.000000
max      2889.000000
dtype: float64

In [20]:
sources = set(df.MentionSourceName)

In [37]:
domains = { ".".join(s.split(".")[1:]) for s in sources }
len(domains)

21

In [29]:
threshold = 10
nodes = []
edges = []
new_sources = sources

for source1 in sources:
    nodes.append({ 'id' : source1, 'size' : int(nb_articles[source1]), 'group' : ".".join(source1.split(".")[1:]) }) 
    events1 = set(df[df.MentionSourceName==source1].GLOBALEVENTID)
    new_sources = new_sources - {source1}
    
    for source2 in new_sources:
        if source1==source2:
            continue
            
        events2 = set(df[df.MentionSourceName==source2].GLOBALEVENTID)
        w = len(events1 & events2)
        
        if w > threshold:
            edges.append({'source': source1, 'target': source2, 'weight': w})  

In [30]:
nodes

[{'id': 'haberler.com', 'size': 747, 'group': 'com'},
 {'id': 'dw.com', 'size': 564, 'group': 'com'},
 {'id': 'washingtonpost.com', 'size': 497, 'group': 'com'},
 {'id': 'yahoo.com', 'size': 1759, 'group': 'com'},
 {'id': 'wkrb13.com', 'size': 458, 'group': 'com'},
 {'id': 'regnum.ru', 'size': 683, 'group': 'ru'},
 {'id': 'ziarelive.ro', 'size': 1480, 'group': 'ro'},
 {'id': 'reuters.com', 'size': 2404, 'group': 'com'},
 {'id': 'nzherald.co.nz', 'size': 443, 'group': 'co.nz'},
 {'id': 'ria.ru', 'size': 565, 'group': 'ru'},
 {'id': 'tribunnews.com', 'size': 1501, 'group': 'com'},
 {'id': 'wickedlocal.com', 'size': 617, 'group': 'com'},
 {'id': 'reliefweb.int', 'size': 378, 'group': 'int'},
 {'id': 'zazoom.it', 'size': 1274, 'group': 'it'},
 {'id': 'xinhuanet.com', 'size': 591, 'group': 'com'},
 {'id': 'zocalo.com.mx', 'size': 371, 'group': 'com.mx'},
 {'id': 'sfgate.com', 'size': 537, 'group': 'com'},
 {'id': 'uol.com.br', 'size': 597, 'group': 'com.br'},
 {'id': 'bbc.com', 'size': 418,

In [31]:
len(edges)

87

In [32]:
network = {'nodes': nodes, 'edges': edges}

In [33]:
with open("data/network.json", "w") as f:
    json.dump(network, f, indent=2)